# Abstract

In this project, we search for the optical configuration of a gravitational wave interferometer optimised for the detection of gravitational waves emitted by the post-merger remnants of binary neutron star (BNS) mergers. When the post-merger remnant is a neutron star, its extreme temperature and density provide an unparalleled cosmic laboratory to explore the (hot) nuclear equation of state. We numerically model many possible detector designs by varying optical parameters and inject numerical relativity waveforms of BNS mergers to calculate the expected detection rate of post-merger signals. We find that the configuration maximising this detection rate improves on the calculated detection rate for NEMO. These results are only preliminary and there is much more work to be done (see all the TODOs scattered throughout these notebooks). Nonetheless, I hope that the reader finds the notebook a stepping stone towards bringing NEMO to life. 

A lot of the learning process involved in understanding gravitational wave interferometery was spread throughout my daily notebooks and is not recorded directly in these results. I am more than happy to take questions via e-mail at juyu0917@uni.sydney.edu.au.

# Acknowledgements

A huge thank you to Bram Slagmolen from CGA for supervising this project over the summer. The thanks extends to Vaishali Adya for her helpful insights. Also thanks to Ling Sun for her guidance on tying the interferometer modeling with the astrophysics. This work builds heavily on the work of previous students of CGA including Mallika Sinha, Lewis Miller, and Glenn de Vine. The python packages Finesse and bilby are utilised often and my thanks to the teams maintaining them.

# 1. Finesse3 Interface (Interactive)  
Enter in parameters and click 'Run Interact' to see sensitivity curves (interactive in plotly). The console will print out properties of the first sensitivity peak found and power detectors placed throughout the detector. Edit the ``save_path`` variable and click 'Save' to save a .txt file of the curve (ASD with radiation pressure). When the ITM transmission ``itmT`` is changed, the PRM transmission ``prmT`` is optimised to maximise the arm cavity power. The laser power (default is 500W) is then adjusted so that the arm cavity power matches the nominal value of 4.5MW (ideally, the required laser power is as low as possible).  
The vSRM is a coupled Michelson and acts as a variable reflectivity signal recycling mirror. Approximately, we have that the SRM tuning and transmission are related to the vSRM common and differential mode tuning, respectively: $\phi_\text{SRC}\simeq-90^\circ+\phi_\text{comm}$ and $T_\text{SRM}\simeq 1-\cos^2(2\phi_\text{diff}\cdot\frac{\pi}{180})$ (we verify this in Section 3).  
The .txt file naming convention for SRM curves is: ``SRM_srmT_srmPhi_srcL_itmT_prmT_lasPow_ASD_with_RP.txt``

The .txt file naming convention for vSRM curves is: ``vSRM_phiComm_phiDiff_srcL_itmT_prmT_lasPow_ASD_with_RP.txt``

## 1.1 SRM Configuration Dashboard

In [2]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

save_path = "./SensitivityCurves/"
default_srmT = 0.04827586206896554
default_srmPhi = -90.02274126019697
default_srcL = 354
default_itmT = 0.01397

store_prmT = 0
store_lasPow = 0
store_itmT = 0

def plot(srmT,srmPhi,srcL,itmT): # This executes when 'Interact' is clicked    
    global store_prmT
    global store_lasPow
    global store_itmT
    
    # Initialise variables (
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(100,10e3,201)
    peak_sens = 0
    peak_f = 0
    peak_bw = 0
    left_f = 0
    right_f = 0
    
    srmT = float(srmT)
    srmPhi = float(srmPhi)
    srcL = float(srcL)
    itmT = float(itmT)
    
    # If itmT changes, then optimise prmT and lasPow
    if itmT != store_itmT:
        peakPow, prmT, lasPow = find_optimal_prmT(srmT,srmPhi,srcL,itmT)
        store_prmT = prmT
        store_lasPow = lasPow
        store_itmT = itmT
    else:
        prmT = store_prmT
        lasPow = store_lasPow
    
    kat = finesse.Model()
    # From SRC_tunability_DCreadout_changedBW_fin3exp.ipynb
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   SRM
    ###########################################################################
    s src bs.p4 srm.p1 L={srcL}
    m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

    # CHECK SRC power
    pd P_SRC srm.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(srm.p2, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    sq sqz db=7 angle=90
    link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take reciprocal sensitivity to use findpeaks i.e. FWHM defined by using reciprocal sensitivity!
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(201), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f
    
    fig_qnoise = go.Figure()
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_with_RP']),mode='lines+markers',name='qnoised NSR'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_without_RP']),mode='lines+markers',name='qshot NSR'))
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.add_vline(x=peak_f)
    fig_qnoise.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_qnoise.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_qnoise.update_layout(title="ASD (qnoised, qshot)",xaxis_title="Frequency [Hz]",yaxis_title="Sensitivity [1/rt Hz]")
    fig_qnoise.show()
    
    fig_signal = go.Figure()
    fig_signal.add_trace(go.Scatter(x=fsig, y=np.abs(out['signal']),mode='lines+markers'))
    fig_signal.update_xaxes(type="log")
    fig_signal.update_yaxes(type="log")
    fig_signal.add_vline(x=peak_f)
    fig_signal.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_signal.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_signal.update_layout(title="Signal Gain (pd1)",xaxis_title="Frequency [Hz]",yaxis_title="Power [W]")
    fig_signal.show()
    
    # Print model outputs
    print(f"Optimal prmT: {prmT}")
    print(f"Peak Sensitivity: {peak_sens} 1/rt Hz, Peak Frequency: {peak_f}Hz, Peak FWHM: {peak_bw}Hz")
    print(f"Input laser power: {np.max(np.abs(out['P_in']))}W")
    print(f"PRC power: {np.max(np.abs(out['P_PRC']))*1e-3}kW")
    print(f"Laser power incident on BS: {np.max(np.abs(out['P_BS']))*1e-3}kW")
    print(f"X-arm cavity power: {np.max(np.abs(out['P_armX']))*1e-6}MW")
    print(f"Y-arm cavity power: {np.max(np.abs(out['P_armY']))*1e-6}MW")
    print(f"SRC power: {np.max(np.abs(out['P_SRC']))}W")
    
    def save_curve(arg):
        lines = [f"{fsig[i]} {np.abs(out['NSR_with_RP'])[i]}\n" for i in range(201)]
        lines[-1].rstrip('\n')
        filename = f"SRM_{srmT}_{srmPhi}_{srcL}_{itmT}_{prmT}_{lasPow}_ASD_with_RP.txt"
        file = open(save_path+filename, "w")
        file.writelines(lines)
        file.close()
        print(f"Saved {filename}!")

    button_save = widgets.Button(description = 'Save')   
    button_save.on_click(save_curve)
    display(button_save)

# Default values here
widgets.interact_manual(plot, srmT=widgets.Text(value=f"{default_srmT}"),
                 srmPhi=widgets.Text(value=f"{default_srmPhi}"), srcL=widgets.Text(value=f"{default_srcL}"), itmT=widgets.Text(value=f"{default_itmT}"));

# Auto-tune prmT with findpeaks to maximise arm cavity power (impedance matching)
def find_optimal_prmT(srmT,srmPhi,srcL,itmT):
    armPow = 4.5e6
    lasPow = 500
    
    vary_prmT = np.geomspace(0.01,0.5,100)
    circX = np.zeros((100,))
    # Find prmT to maximise arm cavity power
    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
        f"""
        # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs R=0.4999625 T=0.4999625 alpha=45

        # CHECK Input laser power
        # pd P_in L0.p1.o
        # CHECK Laser power incident on BS
        # pd P_BS bs.p1.i
        # CHECK PRC Power
        # pd P_PRC bs.p1.o

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=20u phi=179.99999

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=20u phi=90.00001

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        # CHECK Y-arm cavity power
        # pd P_armY etmy.p1.i

        ###########################################################################
        ###   SRM
        ###########################################################################
        s src bs.p4 srm.p1 L={srcL}
        m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

        # CHECK SRC power
        # pd P_SRC srm.p1.i

        ###########################################################################
        ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
        ###########################################################################
        noxaxis()
        """
        )
        out = kat.run()
        circX[i] = out['P_armX']
    
    peak_power = circX[np.argmax(circX)]
    peak_T = vary_prmT[np.argmax(circX)]
    
    if peak_power < armPow:
        while peak_power < armPow and lasPow < 500*1.10:
            lasPow += 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   SRM
            ###########################################################################
            s src bs.p4 srm.p1 L={srcL}
            m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

            # CHECK SRC power
            # pd P_SRC srm.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
    else: 
        while peak_power > armPow:
            lasPow -= 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   SRM
            ###########################################################################
            s src bs.p4 srm.p1 L={srcL}
            m srm R={0.99985-srmT} T={srmT} phi={srmPhi}

            # CHECK SRC power
            # pd P_SRC srm.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
        lasPow += 1
    
    return peak_power, peak_T, lasPow


interactive(children=(Text(value='0.04827586206896554', description='srmT'), Text(value='-90.02274126019697', …

## 1.2 vSRM Configuration Dashboard  


In [1]:
import finesse
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy import interpolate

save_path = "./SensitivityCurves/"
default_phiComm = 0
default_phiDiff = 6.3565
default_srcL = 354
default_itmT = 0.01397

store_prmT = 0
store_lasPow = 0
store_itmT = 0

def plot(phiComm,phiDiff,srcL,itmT): # This executes when 'Interact' is clicked    
    global store_prmT
    global store_lasPow
    global store_itmT
    
    # Initialise variables (
    # if no peak is found a log error will be thrown for the maximum and half-maximum lines but the sensitivity curves are OK)
    fsig = np.geomspace(100,10e3,201)
    peak_sens = 0
    peak_f = 0
    peak_bw = 0
    left_f = 0
    right_f = 0
    
    phiComm = float(phiComm)
    phiDiff = float(phiDiff)
    srcL = float(srcL)
    itmT = float(itmT)
    
    # If itmT changes, then optimise prmT and lasPow
    if itmT != store_itmT:
        peakPow, prmT, lasPow = find_optimal_prmT(phiComm,phiDiff,srcL,itmT)
        store_prmT = prmT
        store_lasPow = lasPow
        store_itmT = itmT
    else:
        prmT = store_prmT
        lasPow = store_lasPow
    
    kat = finesse.Model()
    # From SRC_tunability_DCreadout_changedBW_fin3exp.ipynb
    kat.parse(
    f"""
    # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 4000
    var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
    var itmT {itmT}
    var lmichx 4.5
    var lmichy 4.45

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 {lasPow}

    s l_in L0.p1 prm.p1
    # Power recycling mirror
    m prm T={prmT} L=2e-05 phi=90
    s prc prm.p2 bs.p1 L=53

    # Central beamsplitter
    bs bs R=0.4999625 T=0.4999625 alpha=45

    # CHECK Input laser power
    pd P_in L0.p1.o
    # CHECK Laser power incident on BS
    pd P_BS bs.p1.i
    # CHECK PRC Power
    pd P_PRC bs.p1.o

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmxar.p1 L=lmichx

    m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thick itmxar.p2 itmx.p1 L=0
    m itmx T=itmT L=20u phi=180
    s LX itmx.p2 etmx.p1 L=Larm

    m etmx T=5u L=20u phi=179.99999

    pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
    pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

    # CHECK X-arm cavity power
    pd P_armX etmx.p1.i

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmyar.p1 L=lmichy

    m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
    s ar_thicky itmyar.p2 itmy.p1 L=0
    m itmy T=itmT L=20u phi=90
    s LY itmy.p2 etmy.p1 L=Larm

    m etmy T=5u L=20u phi=90.00001

    pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
    pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

    # CHECK Y-arm cavity power
    pd P_armY etmy.p1.i

    ###########################################################################
    ###   vSRM
    ###########################################################################
    s src bs.p4 SRC_BS.p1 L={srcL}
    bs SRC_BS T=0.5 L=0 alpha=45
    s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
    m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
    s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
    m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

    # CHECK SRC power
    pd P_SRC SRC_BS.p1.i

    ###########################################################################
    ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
    ###########################################################################
    dbs OFI 
    link(SRC_BS.p4, OFI.p1)
    readout_dc AS OFI.p3.o

    # A squeezed source could be injected into the dark port
    sq sqz db=-7 angle=90
    link(sqz, OFI.p2)

    # ------------------------------------------------------------------------------
    # Degrees of Freedom
    # ------------------------------------------------------------------------------
    dof STRAIN LX.dofs.h +1  LY.dofs.h -1

    # signal generator
    sgen sig STRAIN

    qnoised NSR_with_RP AS.p1.i nsr=True
    qshot NSR_without_RP AS.p1.i nsr=True
    pd1 signal AS.p1.i f=fsig

    fsig(1)
    xaxis(fsig, log, 100, 10k, 200)
    """
    )
    out = kat.run()
    neg_sens = -np.abs(out['NSR_with_RP']) # Take reciprocal sensitivity to use findpeaks i.e. FWHM defined by using reciprocal sensitivity!
    peak_idxs, _ = find_peaks(neg_sens)
    
    if peak_idxs.size != 0: # If a peak is found (first peak taken)
        fwhm_idxs = peak_widths(neg_sens, peak_idxs, rel_height=0.5)
        left_idx = fwhm_idxs[2][0]
        right_idx = fwhm_idxs[3][0]
        interp_fsig = interpolate.interp1d(np.arange(201), fsig)
        
        left_f = interp_fsig(left_idx)
        right_f = interp_fsig(right_idx)
        peak_sens = np.abs(out['NSR_with_RP'])[peak_idxs[0]]
        peak_f = fsig[peak_idxs[0]]
        peak_bw = right_f - left_f
    
    fig_qnoise = go.Figure()
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_with_RP']),mode='lines+markers',name='qnoised NSR'))
    fig_qnoise.add_trace(go.Scatter(x=fsig, y=np.abs(out['NSR_without_RP']),mode='lines+markers',name='qshot NSR'))
    fig_qnoise.update_xaxes(type="log")
    fig_qnoise.update_yaxes(type="log")
    fig_qnoise.add_vline(x=peak_f)
    fig_qnoise.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_qnoise.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_qnoise.update_layout(title="ASD (qnoised, qshot)",xaxis_title="Frequency [Hz]",yaxis_title="Sensitivity [1/rt Hz]")
    fig_qnoise.show()
    
    fig_signal = go.Figure()
    fig_signal.add_trace(go.Scatter(x=fsig, y=np.abs(out['signal']),mode='lines+markers'))
    fig_signal.update_xaxes(type="log")
    fig_signal.update_yaxes(type="log")
    fig_signal.add_vline(x=peak_f)
    fig_signal.add_vline(x=right_f,line_dash='dash',line_color='green')
    fig_signal.add_vline(x=left_f,line_dash='dash',line_color='green')
    fig_signal.update_layout(title="Signal Gain (pd1)",xaxis_title="Frequency [Hz]",yaxis_title="Power [W]")
    fig_signal.show()
    
    # Print model outputs
    print(f"Optimal prmT: {prmT}")
    print(f"Peak Sensitivity: {peak_sens} 1/rt Hz, Peak Frequency: {peak_f}Hz, Peak FWHM: {peak_bw}Hz")
    print(f"Input laser power: {np.max(np.abs(out['P_in']))}W")
    print(f"PRC power: {np.max(np.abs(out['P_PRC']))*1e-3}kW")
    print(f"Laser power incident on BS: {np.max(np.abs(out['P_BS']))*1e-3}kW")
    print(f"X-arm cavity power: {np.max(np.abs(out['P_armX']))*1e-6}MW")
    print(f"Y-arm cavity power: {np.max(np.abs(out['P_armY']))*1e-6}MW")
    print(f"SRC power: {np.max(np.abs(out['P_SRC']))}W")
    
    def save_curve(arg):
        lines = [f"{fsig[i]} {np.abs(out['NSR_with_RP'])[i]}\n" for i in range(201)]
        lines[-1].rstrip('\n')
        filename = f"vSRM_{phiComm}_{phiDiff}_{srcL}_{itmT}_{prmT}_{lasPow}_ASD_with_RP.txt"
        file = open(save_path+filename, "w")
        file.writelines(lines)
        file.close()
        print(f"Saved {filename}!")

    button_save = widgets.Button(description = 'Save')   
    button_save.on_click(save_curve)
    display(button_save)

# Default values here
widgets.interact_manual(plot, phiComm=widgets.Text(value=f"{default_phiComm}"),
                 phiDiff=widgets.Text(value=f"{default_phiDiff}"), srcL=widgets.Text(value=f"{default_srcL}"), itmT=widgets.Text(value=f"{default_itmT}"));

# Auto-tune prmT with findpeaks to maximise arm cavity power (impedance matching)
def find_optimal_prmT(phiComm,phiDiff,srcL,itmT):
    armPow = 4.5e6
    lasPow = 500
    
    vary_prmT = np.geomspace(0.01,0.5,100)
    circX = np.zeros((100,))
    # Find prmT to maximise arm cavity power
    for i, prmT in enumerate(vary_prmT):
        kat = finesse.Model()
        kat.parse(
        f"""
        # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
        ###########################################################################
        ###   Variables
        ###########################################################################
        var Larm 4000
        var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
        var itmT {itmT}
        var lmichx 4.5
        var lmichy 4.45

        ###########################################################################
        ###   Input optics
        ###########################################################################
        l L0 500

        s l_in L0.p1 prm.p1
        # Power recycling mirror
        m prm T={prmT} L=2e-05 phi=90
        s prc prm.p2 bs.p1 L=53

        # Central beamsplitter
        bs bs R=0.4999625 T=0.4999625 alpha=45

        # CHECK Input laser power
        # pd P_in L0.p1.o
        # CHECK Laser power incident on BS
        # pd P_BS bs.p1.i
        # CHECK PRC Power
        # pd P_PRC bs.p1.o

        ###########################################################################
        ###   X arm
        ###########################################################################
        s lx bs.p3 itmxar.p1 L=lmichx

        m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thick itmxar.p2 itmx.p1 L=0
        m itmx T=itmT L=20u phi=180
        s LX itmx.p2 etmx.p1 L=Larm

        m etmx T=5u L=20u phi=179.99999

        pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
        pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

        # CHECK X-arm cavity power
        pd P_armX etmx.p1.i

        ###########################################################################
        ###   Y arm
        ###########################################################################
        s ly bs.p2 itmyar.p1 L=lmichy

        m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
        s ar_thicky itmyar.p2 itmy.p1 L=0
        m itmy T=itmT L=20u phi=90
        s LY itmy.p2 etmy.p1 L=Larm

        m etmy T=5u L=20u phi=90.00001

        pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
        pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

        # CHECK Y-arm cavity power
        # pd P_armY etmy.p1.i

        ###########################################################################
        ###   vSRM
        ###########################################################################
        s src bs.p4 SRC_BS.p1 L={srcL}
        bs SRC_BS T=0.5 L=0 alpha=45
        s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
        m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
        s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
        m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

        # CHECK SRC power
        # pd P_SRC SRC_BS.p1.i

        ###########################################################################
        ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
        ###########################################################################
        noxaxis()
        """
        )
        out = kat.run()
        circX[i] = out['P_armX']
    
    peak_power = circX[np.argmax(circX)]
    peak_T = vary_prmT[np.argmax(circX)]
    
    if peak_power < armPow and lasPow < 500*1.10:
        while peak_power < armPow:
            lasPow += 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   vSRM
            ###########################################################################
            s src bs.p4 SRC_BS.p1 L={srcL}
            bs SRC_BS T=0.5 L=0 alpha=45
            s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
            m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
            s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
            m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

            # CHECK SRC power
            # pd P_SRC SRC_BS.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
    else: 
        while peak_power > armPow:
            lasPow -= 1
            kat = finesse.Model()
            kat.parse(
            f"""
            # NEMO Base Model (simplified from OzHF_ITMloss_4km.kat and converted to Finesse 3)
            ###########################################################################
            ###   Variables
            ###########################################################################
            var Larm 4000
            var Mtm  74.1 # from NEMO paper (94.4 in OzHF_ITMloss_4km.kat)
            var itmT {itmT}
            var lmichx 4.5
            var lmichy 4.45

            ###########################################################################
            ###   Input optics
            ###########################################################################
            l L0 {lasPow}

            s l_in L0.p1 prm.p1
            # Power recycling mirror
            m prm T={peak_T} L=2e-05 phi=90
            s prc prm.p2 bs.p1 L=53

            # Central beamsplitter
            bs bs R=0.4999625 T=0.4999625 alpha=45

            # CHECK Input laser power
            # pd P_in L0.p1.o
            # CHECK Laser power incident on BS
            # pd P_BS bs.p1.i
            # CHECK PRC Power
            # pd P_PRC bs.p1.o

            ###########################################################################
            ###   X arm
            ###########################################################################
            s lx bs.p3 itmxar.p1 L=lmichx

            m itmxar T=1-265.0e-06 L=265.0e-06 phi=180 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thick itmxar.p2 itmx.p1 L=0
            m itmx T=itmT L=20u phi=180
            s LX itmx.p2 etmx.p1 L=Larm

            m etmx T=5u L=20u phi=179.99999

            pendulum itmx_sus itmx.mech mass=Mtm fz=1 Qz=1M
            pendulum etmx_sus etmx.mech mass=Mtm fz=1 Qz=1M

            # CHECK X-arm cavity power
            pd P_armX etmx.p1.i

            ###########################################################################
            ###   Y arm
            ###########################################################################
            s ly bs.p2 itmyar.p1 L=lmichy

            m itmyar T=1-265.0e-06 L=265.0e-06 phi=90 # phi from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb (0.0 in OzHF_ITMloss_4km.kat)
            s ar_thicky itmyar.p2 itmy.p1 L=0
            m itmy T=itmT L=20u phi=90
            s LY itmy.p2 etmy.p1 L=Larm

            m etmy T=5u L=20u phi=90.00001

            pendulum itmy_sus itmy.mech mass=Mtm fz=1 Qz=1M
            pendulum etmy_sus etmy.mech mass=Mtm fz=1 Qz=1M

            # CHECK Y-arm cavity power
            # pd P_armY etmy.p1.i

            ###########################################################################
            ###   vSRM
            ###########################################################################
            s src bs.p4 SRC_BS.p1 L={srcL}
            bs SRC_BS T=0.5 L=0 alpha=45
            s vSRC1 SRC_BS.p2 vSRM1.p1 L=4.5
            m vSRM1 T=0 L=0 phi={-90+phiComm+phiDiff}
            s vSRC2 SRC_BS.p3 vSRM2.p1 L=4.5
            m vSRM2 T=0 L=0 phi={0+phiComm-phiDiff}

            # CHECK SRC power
            # pd P_SRC SRC_BS.p1.i

            ###########################################################################
            ###   Output & squeezing (from SRC_tunability_DCreadout_changedBW_fin3exp.ipynb)
            ###########################################################################
            noxaxis()
            """
            )
            out = kat.run()
            peak_power = out['P_armX']
        lasPow += 1
    
    return peak_power, peak_T, lasPow


interactive(children=(Text(value='0', description='phiComm'), Text(value='6.3565', description='phiDiff'), Tex…